# Data cleaning

**data_understanding.ipynb** gir et inntrykk av at datasettet fra Norsk Klimaservicesenter (KSS) inneholder noen manglende verdier spesielt for "snø", og har ellers en liten andel av duplikater og et lite standardavvik. Altså kan rensing av data virke noe overflødig. For å vise intuisjonen til data rensing, vil både det orginale datasettet (blindern.csv) og en modifisert versjon av orginalen fra KSS bli renset. Det modifiserte datasettet vil bli kalt **blindern_dirty_data_generator.csv** og vil ha en større andel av feil og mangler i datasettet i motsetning til hva orginalen hadde.


### Rensing av blindern.csv

In [3]:
import pandas as pd
import numpy as np
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../src')))

from data_preparation.data_cleaning import DataCleaning

# Original file - "blindern.csv"

original_file = "blindern.csv"
data_path = os.path.join(os.getcwd(),"..", "data")
original_path = os.path.join(data_path, original_file)  
df = pd.read_csv(original_path, delimiter=";")
data_cleaner = DataCleaning(df)

modified_file_original = os.path.join(data_path, "blindern_data_cleaning.csv")  

# Cleaning original file
df = data_cleaner.form().duplicates().missing().outliers().replacing().df

# Save the modified DataFrame as a new CSV file in the same folder
df.to_csv(modified_file_original, index=False, sep=";")  # Keep the same delimiter (;)

print(f"Modified CSV file saved as: {modified_file_original}")

Modified CSV file saved as: c:\Users\livem\OneDrive - NTNU\Dokumenter\Andvendt programmering\Miljo_dataanalyse_applikasjon\notebooks\..\data\blindern_data_cleaning.csv


### Rensing av blindern_dirty_data_generator.csv

In [4]:
import pandas as pd
import numpy as np
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../src')))

from data_preparation.data_cleaning import DataCleaning

# Dirty file - "blindern_dirty_data_generator.csv"

dirty_file = "blindern_dirty_data_generator.csv"  
data_path = os.path.join(os.getcwd(),"..", "data")
dirty_path = os.path.join(data_path, dirty_file)
df_dirty = pd.read_csv(dirty_path, delimiter=";")

data_cleaner = DataCleaning(df_dirty)
df_dirty = data_cleaner.form().duplicates().missing().outliers().replacing().df

# Save the modified DataFrame as a new CSV file in the same folder
modified_file_dirty = os.path.join(data_path, "blindern_dirty_data_cleaning.csv")  
df_dirty.to_csv(modified_file_dirty, index=False, sep=";")  # Keep the same delimiter (;)

print(f"Modified CSV file saved as: {modified_file_dirty}")


Modified CSV file saved as: c:\Users\livem\OneDrive - NTNU\Dokumenter\Andvendt programmering\Miljo_dataanalyse_applikasjon\notebooks\..\data\blindern_dirty_data_cleaning.csv
